> **Copyright (c) 2020 Skymind Holdings Berhad**<br><br>
> **Copyright (c) 2021 Skymind Education Group Sdn. Bhd.**<br>
<br>
Licensed under the Apache License, Version 2.0 (the \"License\");
<br>you may not use this file except in compliance with the License.
<br>You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0/
<br>
<br>Unless required by applicable law or agreed to in writing, software
<br>distributed under the License is distributed on an \"AS IS\" BASIS,
<br>WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
<br>See the License for the specific language governing permissions and
<br>limitations under the License.
<br>
<br>
**SPDX-License-Identifier: Apache-2.0**
<br>

# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

# Notebook Content

* [Topic Modeling 1](#Topic-Modeling-1)
    

* [Topic Modeling 2](#Topic-Modeling-2)


* [Topic Modeling 3](#Topic-Modeling-3)


* [Identify Topics in Each Document](#Identify-Topics-in-Each-Document)


* [Additional Exercises](#Additional-Exercises)

## Topic Modeling 1

### Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('models/dtm_stop.pkl')
data

aaaaah  aaaaahhhhhhh  aaaaauuugghhhhhh  aaaahhhhh  aaah  aah  abc  \
ali           0             0                 0          0     0    0    1   
anthony       0             0                 0          0     0    0    0   
bill          1             0                 0          0     0    0    0   
bo            0             1                 1          1     0    0    0   
dave          0             0                 0          0     1    0    0   
hasan         0             0                 0          0     0    0    0   
jim           0             0                 0          0     0    0    0   
joe           0             0                 0          0     0    0    0   
john          0             0                 0          0     0    0    0   
louis         0             0                 0          0     0    3    0   
mike          0             0                 0          0     0    0    0   
ricky         0             0                 0          0     0    0    0   

         abcs  ability  abject  ...  zee  zen  zeppelin  zero  zillion  \
ali         0        0       0  ...    0    0         0     0        0   
anthony     0        0       0  ...    0    0         0     0        0   
bill        1        0       0  ...    0    0         0     1        1   
bo          0        1       0  ...    0    0         0     1        0   
dave        0        0       0  ...    0    0         0     0        0   
hasan       0        0       0  ...    2    1         0     1        0   
jim         0        0       0  ...    0    0         0     0        0   
joe         0        0       0  ...    0    0         0     0        0   
john        0        0       0  ...    0    0         0     0        0   
louis       0        0       0  ...    0    0         0     2        0   
mike        0        0       0  ...    0    0         2     1        0   
ricky       0        1       1  ...    0    0         0     0        0   

         zombie  zombies  zoning  zoo  éclair  
ali           1        0       0    0       0  
anthony       0        0       0    0       0  
bill          1        1       1    0       0  
bo            0        0       0    0       0  
dave          0        0       0    0       0  
hasan         0        0       0    0       0  
jim           0        0       0    0       0  
joe           0        0       0    0       0  
john          0        0       0    0       1  
louis         0        0       0    0       0  
mike          0        0       0    0       0  
ricky         0        0       0    1       0  

[12 rows x 7468 columns]

In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\tanch\anaconda3\envs\nlp-labs\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

ali  anthony  bill  bo  dave  hasan  jim  joe  john  louis  \
aaaaah              0        0     1   0     0      0    0    0     0      0   
aaaaahhhhhhh        0        0     0   1     0      0    0    0     0      0   
aaaaauuugghhhhhh    0        0     0   1     0      0    0    0     0      0   
aaaahhhhh           0        0     0   1     0      0    0    0     0      0   
aaah                0        0     0   0     1      0    0    0     0      0   

                  mike  ricky  
aaaaah               0      0  
aaaaahhhhhhh         0      0  
aaaaauuugghhhhhh     0      0  
aaaahhhhh            0      0  
aaah                 0      0

In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("models/cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.008*"fucking" + 0.006*"say" + 0.005*"going" + 0.005*"shit" + 0.005*"want" + 0.005*"fuck" + 0.005*"didnt" + 0.005*"went" + 0.005*"day" + 0.005*"hes"'),
 (1,
  '0.006*"cause" + 0.006*"shit" + 0.006*"theyre" + 0.006*"fuck" + 0.005*"theres" + 0.005*"really" + 0.005*"fucking" + 0.005*"life" + 0.004*"hes" + 0.004*"thing"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.007*"say" + 0.006*"going" + 0.006*"dad" + 0.005*"fucking" + 0.005*"want" + 0.004*"shes" + 0.004*"went" + 0.004*"hes" + 0.004*"didnt" + 0.004*"love"'),
 (1,
  '0.006*"shit" + 0.005*"hes" + 0.005*"life" + 0.005*"love" + 0.005*"ok" + 0.004*"cause" + 0.004*"thing" + 0.004*"good" + 0.004*"theres" + 0.004*"really"'),
 (2,
  '0.009*"fucking" + 0.007*"shit" + 0.007*"fuck" + 0.005*"say" + 0.005*"theyre" + 0.005*"cause" + 0.005*"didnt" + 0.005*"going" + 0.005*"day" + 0.005*"thing"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.010*"dad" + 0.007*"going" + 0.007*"want" + 0.006*"love" + 0.005*"shes" + 0.005*"hasan" + 0.005*"say" + 0.004*"life" + 0.004*"mom" + 0.004*"hey"'),
 (1,
  '0.010*"fucking" + 0.007*"shit" + 0.007*"theyre" + 0.007*"fuck" + 0.006*"say" + 0.006*"hes" + 0.006*"thing" + 0.006*"theres" + 0.005*"little" + 0.005*"didnt"'),
 (2,
  '0.006*"day" + 0.006*"fucking" + 0.006*"cause" + 0.006*"went" + 0.005*"really" + 0.005*"goes" + 0.005*"going" + 0.005*"say" + 0.005*"thing" + 0.004*"good"'),
 (3,
  '0.007*"fucking" + 0.006*"shit" + 0.006*"fuck" + 0.006*"man" + 0.005*"good" + 0.005*"didnt" + 0.005*"love" + 0.005*"want" + 0.005*"ahah" + 0.004*"bo"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling 2
### Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [9]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tanch\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [11]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('models/data_clean.pkl')
data_clean

transcript
ali      ladies and gentlemen please welcome to the sta...
anthony  thank you thank you thank you san francisco th...
bill      all right thank you thank you very much thank...
bo       bo what old macdonald had a farm e i e i o and...
dave     this is dave he tells dirty jokes for a living...
hasan      whats up davis whats up im home i had to bri...
jim         ladies and gentlemen please welcome to the ...
joe         ladies and gentlemen welcome joe rogan  wha...
john     all right petunia wish me luck out there you w...
louis    introfade the music out lets roll hold there l...
mike     wow hey thank you thanks thank you guys hey se...
ricky    hello hello how you doing great thank you wow ...

In [12]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

transcript
ali      ladies gentlemen stage ali hi thank hello na s...
anthony  thank thank people i em i francisco city world...
bill     thank thank pleasure georgia area oasis i june...
bo       macdonald farm e i o farm pig e i i snort macd...
dave     jokes living stare work profound train thought...
hasan    whats davis whats home i netflix la york i son...
jim      ladies gentlemen stage mr jim jefferies thank ...
joe      ladies gentlemen joe fuck thanks phone fuckfac...
john     petunia thats hello hello chicago thank crowd ...
louis    music lets lights lights thank i i place place...
mike     wow hey thanks look insane years everyone i id...
ricky    hello thank fuck thank im gon youre weve money...

In [13]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

aaaaahhhhhhh  aaaaauuugghhhhhh  aaaahhhhh  aah  abc  abcs  ability  \
ali                 0                 0          0    0    1     0        0   
anthony             0                 0          0    0    0     0        0   
bill                0                 0          0    0    0     1        0   
bo                  1                 1          1    0    0     0        1   
dave                0                 0          0    0    0     0        0   
hasan               0                 0          0    0    0     0        0   
jim                 0                 0          0    0    0     0        0   
joe                 0                 0          0    0    0     0        0   
john                0                 0          0    0    0     0        0   
louis               0                 0          0    3    0     0        0   
mike                0                 0          0    0    0     0        0   
ricky               0                 0          0    0    0     0        1   

         abortion  abortions  abuse  ...  yummy  ze  zealand  zee  zeppelin  \
ali             0          0      0  ...      0   0        0    0         0   
anthony         2          0      0  ...      0   0       10    0         0   
bill            0          0      0  ...      0   1        0    0         0   
bo              0          0      0  ...      0   0        0    0         0   
dave            0          1      0  ...      0   0        0    0         0   
hasan           0          0      0  ...      0   0        0    1         0   
jim             0          0      0  ...      0   0        0    0         0   
joe             0          0      1  ...      0   0        0    0         0   
john            0          0      0  ...      0   0        0    0         0   
louis           0          0      0  ...      0   0        0    0         0   
mike            0          0      0  ...      0   0        0    0         2   
ricky           0          0      0  ...      1   0        0    0         0   

         zillion  zombie  zombies  zoo  éclair  
ali            0       1        0    0       0  
anthony        0       0        0    0       0  
bill           1       1        1    0       0  
bo             0       0        0    0       0  
dave           0       0        0    0       0  
hasan          0       0        0    0       0  
jim            0       0        0    0       0  
joe            0       0        0    0       0  
john           0       0        0    0       1  
louis          0       0        0    0       0  
mike           0       0        0    0       0  
ricky          0       0        0    1       0  

[12 rows x 4635 columns]

In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"shit" + 0.009*"man" + 0.009*"life" + 0.008*"fuck" + 0.008*"thing" + 0.007*"day" + 0.007*"hes" + 0.007*"lot" + 0.006*"cause" + 0.006*"women"'),
 (1,
  '0.009*"day" + 0.009*"thing" + 0.007*"way" + 0.006*"joke" + 0.006*"cause" + 0.006*"hes" + 0.006*"years" + 0.006*"gon" + 0.005*"guy" + 0.005*"shit"')]

In [16]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.017*"shit" + 0.010*"man" + 0.009*"fuck" + 0.008*"dude" + 0.008*"lot" + 0.008*"gon" + 0.007*"hes" + 0.007*"guy" + 0.006*"thing" + 0.006*"day"'),
 (1,
  '0.007*"day" + 0.007*"way" + 0.007*"dad" + 0.007*"hes" + 0.006*"thing" + 0.006*"life" + 0.006*"shes" + 0.006*"years" + 0.006*"night" + 0.005*"man"'),
 (2,
  '0.013*"thing" + 0.012*"day" + 0.011*"life" + 0.009*"cause" + 0.009*"fuck" + 0.007*"women" + 0.007*"guy" + 0.007*"man" + 0.007*"things" + 0.007*"kids"')]

In [17]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.011*"shit" + 0.010*"man" + 0.010*"thing" + 0.009*"life" + 0.008*"fuck" + 0.008*"way" + 0.008*"hes" + 0.008*"day" + 0.007*"guy" + 0.007*"cause"'),
 (1,
  '0.013*"day" + 0.010*"women" + 0.009*"thing" + 0.009*"lot" + 0.008*"cause" + 0.008*"shit" + 0.006*"man" + 0.006*"fuck" + 0.006*"ass" + 0.006*"guns"'),
 (2,
  '0.009*"cause" + 0.009*"point" + 0.008*"kind" + 0.006*"way" + 0.006*"id" + 0.006*"jenny" + 0.006*"night" + 0.006*"car" + 0.005*"guy" + 0.005*"gon"'),
 (3,
  '0.010*"dad" + 0.008*"hes" + 0.008*"shes" + 0.007*"life" + 0.007*"years" + 0.007*"day" + 0.006*"thing" + 0.006*"joke" + 0.005*"man" + 0.005*"way"')]

## Topic Modeling 3

### Attempt #3 (Nouns and Adjectives)

In [18]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [19]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

transcript
ali      ladies gentlemen welcome stage ali wong hi wel...
anthony  thank san francisco thank good people surprise...
bill     right thank thank pleasure greater atlanta geo...
bo       old macdonald farm e i i o farm pig e i i snor...
dave     dirty jokes living stare most hard work profou...
hasan    whats davis whats im home i netflix special la...
jim      ladies gentlemen welcome stage mr jim jefferie...
joe      ladies gentlemen joe fuck san francisco thanks...
john     right petunia august thats good right hello he...
louis    music lets lights lights thank much i i i nice...
mike     wow hey thanks hey seattle nice look crazy ins...
ricky    hello great thank fuck thank lovely welcome im...

In [20]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

aaaaah  aaaaahhhhhhh  aaaaauuugghhhhhh  aaaahhhhh  aah  abc  abcs  \
ali           0             0                 0          0    0    1     0   
anthony       0             0                 0          0    0    0     0   
bill          1             0                 0          0    0    0     1   
bo            0             1                 1          1    0    0     0   
dave          0             0                 0          0    0    0     0   
hasan         0             0                 0          0    0    0     0   
jim           0             0                 0          0    0    0     0   
joe           0             0                 0          0    0    0     0   
john          0             0                 0          0    0    0     0   
louis         0             0                 0          0    3    0     0   
mike          0             0                 0          0    0    0     0   
ricky         0             0                 0          0    0    0     0   

         ability  abject  able  ...  ze  zealand  zee  zeppelin  zero  \
ali            0       0     2  ...   0        0    0         0     0   
anthony        0       0     0  ...   0       10    0         0     0   
bill           0       0     1  ...   1        0    0         0     0   
bo             1       0     0  ...   0        0    0         0     1   
dave           0       0     0  ...   0        0    0         0     0   
hasan          0       0     1  ...   0        0    2         0     0   
jim            0       0     1  ...   0        0    0         0     0   
joe            0       0     2  ...   0        0    0         0     0   
john           0       0     3  ...   0        0    0         0     0   
louis          0       0     1  ...   0        0    0         0     0   
mike           0       0     0  ...   0        0    0         2     0   
ricky          1       1     2  ...   0        0    0         0     0   

         zillion  zombie  zombies  zoo  éclair  
ali            0       1        0    0       0  
anthony        0       0        0    0       0  
bill           1       1        1    0       0  
bo             0       0        0    0       0  
dave           0       0        0    0       0  
hasan          0       0        0    0       0  
jim            0       0        0    0       0  
joe            0       0        0    0       0  
john           0       0        0    0       1  
louis          0       0        0    0       0  
mike           0       0        0    0       0  
ricky          0       0        0    1       0  

[12 rows x 5587 columns]

In [21]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [22]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"ass" + 0.003*"mom" + 0.003*"wife" + 0.003*"dick" + 0.003*"son" + 0.003*"gun" + 0.003*"dog" + 0.003*"ok" + 0.002*"guns" + 0.002*"clinton"'),
 (1,
  '0.006*"joke" + 0.004*"parents" + 0.003*"comedy" + 0.003*"jokes" + 0.003*"mom" + 0.003*"bo" + 0.003*"hasan" + 0.003*"jenny" + 0.003*"love" + 0.003*"repeat"')]

In [23]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"ahah" + 0.005*"guns" + 0.005*"son" + 0.004*"ass" + 0.004*"gay" + 0.004*"nigga" + 0.003*"girlfriend" + 0.003*"american" + 0.003*"class" + 0.003*"gun"'),
 (1,
  '0.008*"joke" + 0.004*"mom" + 0.004*"jokes" + 0.004*"bo" + 0.003*"clinton" + 0.003*"parents" + 0.003*"repeat" + 0.003*"eye" + 0.003*"anthony" + 0.003*"dog"'),
 (2,
  '0.004*"mom" + 0.004*"parents" + 0.003*"hasan" + 0.003*"door" + 0.003*"jenny" + 0.003*"love" + 0.002*"dick" + 0.002*"religion" + 0.002*"stupid" + 0.002*"sense"')]

In [24]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.000*"mom" + 0.000*"parents" + 0.000*"clinton" + 0.000*"wife" + 0.000*"joke" + 0.000*"friend" + 0.000*"movie" + 0.000*"husband" + 0.000*"ass" + 0.000*"class"'),
 (1,
  '0.006*"joke" + 0.003*"nuts" + 0.003*"hell" + 0.003*"husband" + 0.003*"dead" + 0.003*"ok" + 0.003*"gun" + 0.003*"jenner" + 0.003*"doctor" + 0.003*"ass"'),
 (2,
  '0.005*"mom" + 0.005*"friend" + 0.004*"joke" + 0.004*"parents" + 0.003*"jenny" + 0.003*"clinton" + 0.003*"mad" + 0.003*"wife" + 0.003*"ahah" + 0.003*"door"'),
 (3,
  '0.004*"bo" + 0.004*"hasan" + 0.004*"repeat" + 0.004*"um" + 0.004*"guns" + 0.003*"parents" + 0.003*"eye" + 0.003*"dog" + 0.003*"mom" + 0.003*"contact"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [25]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.007*"mom" + 0.006*"joke" + 0.005*"hasan" + 0.005*"parents" + 0.004*"anthony" + 0.004*"door" + 0.004*"mad" + 0.004*"jokes" + 0.003*"brown" + 0.003*"grandma"'),
 (1,
  '0.005*"clinton" + 0.005*"wife" + 0.004*"mom" + 0.004*"parents" + 0.004*"tit" + 0.004*"dog" + 0.004*"cow" + 0.003*"gun" + 0.003*"jesus" + 0.003*"dick"'),
 (2,
  '0.006*"jenny" + 0.006*"ahah" + 0.005*"friend" + 0.004*"nigga" + 0.004*"gay" + 0.003*"accident" + 0.003*"long" + 0.003*"club" + 0.003*"door" + 0.003*"scrambler"'),
 (3,
  '0.005*"joke" + 0.004*"bo" + 0.004*"ass" + 0.004*"um" + 0.004*"repeat" + 0.004*"guns" + 0.003*"eye" + 0.003*"ok" + 0.003*"contact" + 0.003*"cunt"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [26]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(3, 'ali'),
 (0, 'anthony'),
 (1, 'bill'),
 (3, 'bo'),
 (2, 'dave'),
 (0, 'hasan'),
 (3, 'jim'),
 (0, 'joe'),
 (1, 'john'),
 (1, 'louis'),
 (2, 'mike'),
 (3, 'ricky')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

# Contributors

**Author**
<br>Chee Lam

# References

1. [Natural Language Processing in Python](https://www.youtube.com/watch?v=xvqsFTUsOmc&t=6s)